In [1]:
import pandas as pd
from elasticsearch import Elasticsearch, helpers
import requests
from requests.auth import HTTPBasicAuth
import json
import secrets

In [3]:
def ban_words(terms):
    terms = terms.lower()
    terms = terms.split(' ')
    bans=['rue','avenue','de','la','le','du']
    terms = list(set(terms) - set(bans))
    return ' '.join(terms)

In [4]:
def display_result(hit):
    result = ''
    if(hit['_score'] is not None):
        result = result + str(hit['_score'])+' - '
    if(hit['_source']['siren'] is not None):
        result = result + hit['_source']['siren']+' - '
    if(hit['_source']['nom_complet'] is not None):
        result = result + hit['_source']['nom_complet']+' - '
    if(hit['_source']['geo_adresse'] is not None):
        result = result + hit['_source']['geo_adresse']+' - '
    if(hit['_source']['nombre_etablissements'] is not None):
        result = result + str(hit['_source']['nombre_etablissements'])
    
    return result
    

In [5]:
def is_adress(terms):
    headers = {
        "Content-Type": "application/json"
    }
    body = {
        "q": terms
    }
    r = requests.get('https://api-adresse.data.gouv.fr/search/?q='+terms, headers=headers)
    
    if(r.status_code == 200):
        if(len(r.json()['features']) > 0):
            if(r.json()['features'][0]['properties']['score'] > 0.8):
                return True
            else:
                return False
        else:
            return False
    else:
        return False

In [6]:
def search_es_by_adress(terms):    
    terms = ban_words(terms)
    headers = {
        "Content-Type": "application/json"
    }
    body = {
        "sort" : [
            { "etat_administratif_etablissement.keyword" : "asc" },
            "_score"
        ],
        "query": {
            "multi_match": {
            "query": terms,
            "type": "most_fields",
            "fields": [
                "geo_adresse^10"
                ]
            }
        }
    }
    r = requests.get(secrets.ELK_URL+'siren/_search', auth=HTTPBasicAuth(secrets.ELK_USER,secrets.ELK_PASSWORD), headers=headers, data=json.dumps(body))
    for hit in r.json()['hits']['hits']:
        result = display_result(hit)
        print(result)


In [7]:
def search_es_partial_id(terms, index, prop):
    headers = {
        "Content-Type": "application/json"
    }
    body = {
        "sort" : [
            { "etat_administratif_etablissement.keyword" : "asc" },
            { "nombre_etablissements" : "desc" }
        ],
        "query": {
            "prefix": {
              prop+".keyword": {
                "value": terms
              }
            }
          }
    }
    r = requests.get(secrets.ELK_URL+index+'/_search', auth=HTTPBasicAuth(secrets.ELK_USER,secrets.ELK_PASSWORD), headers=headers, data=json.dumps(body))
    for hit in r.json()['hits']['hits']:
        result = display_result(hit)
        print(result)


In [8]:
def search_es_exact_id(terms, index, prop):
    headers = {
        "Content-Type": "application/json"
    }
    body = {
        "sort" : [
            { "etat_administratif_etablissement.keyword" : "asc" },
            { "nombre_etablissements" : "desc" }
        ],
        "query": {
          "term": {
            prop+".keyword": {
              "value": terms,
              "boost": 1.0
            }
          }
        }
    }
    r = requests.get(secrets.ELK_URL+index+'siren/_search', auth=HTTPBasicAuth(secrets.ELK_USER,secrets.ELK_PASSWORD), headers=headers, data=json.dumps(body))
    for hit in r.json()['hits']['hits']:
        result = display_result(hit)
        print(result)
    return result


In [9]:
def is_id(terms):
    for t in terms.split(' '):
        try:
            int(t)
            if((len(t) < 9) & (len(t) > 6)):
                # recherche partielle
                result = search_es_partial_id(t, 'siren', 'siren')
                if(result != ''):
                    return True
            else:
                if(len(t) == 9):
                    # recherche exact 
                    result = search_es_exact_id(t, 'siren', 'siren')
                    if(result != ''):
                        return True
                else:
                    if((len(t) < 14) & (len(t) > 6)):
                        # recherche partiel siret
                        result = search_es_partial_id(t, 'siret', 'siret')
                        if(result != ''):
                            return True
                    elif(len(t) == 14):
                        # recherche exact siret
                        result = search_es_exact_id(t, 'siret', 'siret')
                        if(result != ''):
                            return True
        except:
            if(t[0] == 'W'):
                try:
                    int(t[1:])
                    if(len(t) == 10):
                        result = search_es_exact_id(t, 'siren', 'identifiantAssociationUniteLegale')
                        if(result != ''):
                            return True
                    elif(len(t) < 9):
                        result = search_es_partial_id(t, 'siren', 'identifiantAssociationUniteLegale')
                        if(result != ''):
                            return True
                except:
                    pass
            pass
    return False


In [10]:
def search_es_by_name(terms):
    terms = ban_words(terms)
    headers = {
        "Content-Type": "application/json"
    }
    body = {
        "sort" : [
            { "etat_administratif_etablissement.keyword" : "asc" },
            "_score"
        ],
        "query":{
            "bool":{
                "should":[{
                    "function_score": {
                        "query": {
                            "bool":{  
                              "should":[  
                                {  
                                  "multi_match":{  
                                    "query":terms,
                                    "type":"phrase",
                                    "fields":[  
                                        "nom_complet^15",
                                        "siren^3",
                                        "siret^3",
                                        "identifiantAssociationUniteLegale^3"
                                    ],
                                    "boost":5
                                  }
                                }
                              ]
                            }
                          },
                        "functions": [{
                            "field_value_factor": {
                                "field": "nombre_etablissements",
                                "factor": 5,
                                "modifier": "sqrt",
                                "missing": 1
                                }
                            }
                        ]
                    }
                },
                {  
                  "multi_match":{  
                    "query":terms,
                    "type":"most_fields",
                    "fields":[  
                        "nom_complet^10",
                    ],
                    "fuzziness" : "AUTO"

                  }
                }
                ]
            }
        }
    }
        
    r = requests.get(secrets.ELK_URL+'siren/_search',
                     auth=HTTPBasicAuth(secrets.ELK_USER,secrets.ELK_PASSWORD),
                     headers=headers, 
                     data=json.dumps(body))
    for hit in r.json()['hits']['hits']:
        result = display_result(hit)
        print(result)
    


In [11]:
def search_es(terms):
    isid = is_id(terms)
    if(not isid):
        print("Identifiant non trouvé")
        isadress = is_adress(terms)
        if(isadress):
            print("Adresse détectée")
            search_es_by_adress(terms)
        else:
            print("Recherche nom")
            search_es_by_name(terms)

#### Recherche term entreprise

In [12]:
search_es("numericite")

Identifiant non trouvé
Recherche nom
1426.6954 - 889108718 - reseau d'echanges sur le numerique des collectivites territoriales du bassin rhonalpin (numericite) - 50 Boulevard Ambroise Croizat 69200 Vénissieux - 2.0
175.74002 - 822766358 - numeri'cite - 9 Rue de Mulhouse 75002 Paris - 2.0
156.2287 - 824372726 - numericore - 2 Rue Bourgon 75013 Paris - 1.0
156.2142 - 514036003 - numericube - 101 Avenue du Général Leclerc 75014 Paris - 3.0
151.87407 - 481686772 - numericat - 84 Rue des Orteaux 75020 Paris - 1.0
151.8526 - 839054699 - numercity - 4 Rue de Marivaux 75002 Paris - 1.0
124.83323 - 840064208 - nutricite-bio - 11 Rue Robert Delaeter 59128 Flers-en-Escrebieux - 3.0
175.72601 - 491942108 - numericity - 21 b Place du Général de Gaulle 24600 Ribérac - 2.0
156.2287 - 829950757 - numericiel - 13 bis Avenue de la Motte-picquet 75007 Paris - 1.0
156.21333 - 820195402 - numericare - 261 Chemin des Vayoux 06810 Auribeau-sur-Siagne - 1.0


In [13]:
search_es("numeri'cite")

Identifiant non trouvé
Recherche nom
4826.4224 - 822766358 - numeri'cite - 9 Rue de Mulhouse 75002 Paris - 2.0
51.94893 - 889108718 - reseau d'echanges sur le numerique des collectivites territoriales du bassin rhonalpin (numericite) - 50 Boulevard Ambroise Croizat 69200 Vénissieux - 2.0
156.2009 - 491942108 - numericity - 21 b Place du Général de Gaulle 24600 Ribérac - 2.0


In [14]:
search_es("financiere ganymede")

Identifiant non trouvé
Recherche nom
7462.0366 - 414971929 - financiere ganymede - Rue Rigoberta Menchu 84000 Avignon - 4.0
188.51324 - 423208180 - ganymede - Route de Gruissan 11100 Narbonne - 1.0
180.70328 - 409716305 - ganymede - 36 Rue Etienne Ganneron 77600 Bussy-Saint-Georges - 1.0
180.70328 - 901723403 - ganymede - 58 Boulevard Aristide Briand 63000 Clermont-Ferrand - 1.0
180.70096 - 880878145 - ganymede - 128 Rue la Boétie 75008 Paris - 1.0
180.70096 - 817883150 - ganymede - 3 Rue de la Gaîté 75014 Paris - 1.0
175.8141 - 803657048 - ganymede - 4 Impasse Bel Air 57970 Yutz - 1.0
170.7807 - 510973431 - ganymede - Traverse des Pénitents Noirs 13600 La Ciotat - 2.0
164.94908 - 402591820 - ganimede - 5 Rue Paul Gaffarel 21850 Saint-Apollinaire - 1.0
158.11334 - 442743209 - ganimede - Rue du Docteur David 88300 Neufchâteau - 1.0


In [15]:
search_es("ganymede")

Identifiant non trouvé
Recherche nom
4848.6216 - 414971929 - financiere ganymede - Rue Rigoberta Menchu 84000 Avignon - 4.0
4462.8955 - 417691474 - sci ganymede - 14 Quai Kléber 67000 Strasbourg - 3.0
4221.2007 - 510973431 - ganymede - Traverse des Pénitents Noirs 13600 La Ciotat - 2.0
3571.9001 - 383657467 - association ganymede - Rue Del Blagaire 34270 Saint-Mathieu-de-Tréviers - 2.0
3349.9763 - 423208180 - ganymede - Route de Gruissan 11100 Narbonne - 1.0
3211.1895 - 409716305 - ganymede - 36 Rue Etienne Ganneron 77600 Bussy-Saint-Georges - 1.0
3211.1895 - 901723403 - ganymede - 58 Boulevard Aristide Briand 63000 Clermont-Ferrand - 1.0
3211.1484 - 880878145 - ganymede - 128 Rue la Boétie 75008 Paris - 1.0
3211.1484 - 817883150 - ganymede - 3 Rue de la Gaîté 75014 Paris - 1.0
3124.3062 - 803657048 - ganymede - 4 Impasse Bel Air 57970 Yutz - 1.0


#### Recherche adresse

In [16]:
search_es("9 place du général beuret")

Identifiant non trouvé
Adresse détectée
224.55118 - 428462998 - aurora bernardez - 9 Place du Général Beuret 75015 Paris - 1.0
224.55118 - 898044664 - pharmacie de la place - 9 Place du Général Beuret 75015 Paris - 1.0
223.82687 - 402218036 - elisabeth dufourg - 9 Place du Général Beuret 75015 Paris - 2.0
223.79312 - 392095667 - ligue regionale ile de france de squash - 9 Place du Général Beuret 75015 Paris - 4.0
222.80934 - 509750097 - amselle-kramer - 9 Place du Général Beuret 75015 Paris - 1.0
222.80934 - 539664789 - les generaux - 9 Place du Général Beuret 75015 Paris - 1.0
222.54243 - 449210434 - dubar cambronne - 9 Place du Général Beuret 75015 Paris - 1.0
222.54243 - 481797082 - ilan krief - 9 Place du Général Beuret 75015 Paris - 1.0
222.54243 - 038732509 - synd.copr.   9 place gal beuret 75015 pa - 9 Place du Général Beuret 75015 Paris - 1.0
195.70537 - 897983284 - meriem siad - Place du Général Beuret 75015 Paris - 1.0


#### Recherche grosse boîte

In [17]:
search_es("lvmh")

Identifiant non trouvé
Recherche nom
7334.5405 - 351612676 - lvmh fashion group france - 2 Rue du Pont Neuf 75001 Paris - 16.0
7020.9077 - 572082253 - lvmh fragrance brands - 77 Rue Anatole France 92300 Levallois-Perret - 11.0
5617.494 - 322670928 - lvmh recherche - 185 Avenue de Verdun 45800 Saint-Jean-de-Braye - 5.0
5439.0337 - 351524921 - lvmh montres et joaillerie france - 79 Boulevard Haussmann 75008 Paris - 10.0
5111.8647 - 380097881 - lvmh miscellanees - 24 Rue Jean Goujon 75008 Paris - 4.0
4443.1885 - 328478094 - lvmh services - 24 Rue Jean Goujon 75008 Paris - 3.0
4201.9194 - 821312659 - lvmh swiss manufactures sa - 5.0
3768.4006 - 429651441 - lvmh fashion group services - 24 Rue Jean Goujon 75008 Paris - 4.0
3715.2214 - 802046185 - lvmh fashion group support - 24 Rue Jean Goujon 75008 Paris - 4.0
3230.3247 - 502888589 - lvmh metiers d art - 24 Rue Jean Goujon 75008 Paris - 3.0


In [18]:
search_es("total")

Identifiant non trouvé
Recherche nom
10225.118 - 780060406 - total reunion - 3 Avenue Jacques Prévert 97420 Le Port - 29.0
6767.823 - 538612458 - total solar f - 1 Passerelle des Reflets 92400 Courbevoie - 18.0
6591.5474 - 542051180 - totalenergies se (total) - 2 Place Jean Millier 92400 Courbevoie - 16.0
6551.6333 - 054389002 - total mayotte - Lotissement les Trois Vallees - Kaweni 97600 Mamoudzou - 12.0
4794.3047 - 421328295 - total immersion - 33 Rue du Mail 75002 Paris - 6.0
4684.31 - 508393980 - total flex (bhc) - 1 Place des Reflets 92400 Courbevoie - 8.0
4603.8403 - 493085518 - total invest lyon - Chemin du Puits Villefranche-de-Rouergue - 8.0
4558.174 - 582102091 - total consortium clayton - 31 Rue Buffon 75005 Paris - 8.0
4311.232 - 838517621 - total wellness - 625 Rte des Marais 73790 Tours-en-Savoie - 5.0
3939.476 - 501035331 - total renov - 10 Rue de Chantemerle 77590 Bois-le-Roi - 4.0


#### Recherche sirene

In [19]:
search_es("414971929")

Identifiant non trouvé
Recherche nom
987.99896 - 414971929 - financiere ganymede - Rue Rigoberta Menchu 84000 Avignon - 4.0


#### Recherche siren partiel 

In [20]:
search_es("4149719")

414971929 - financiere ganymede - Rue Rigoberta Menchu 84000 Avignon - 4.0
414971978 - sci vermeer - Impasse Castel 83000 Toulon - 2.0
414971937 - anthony lepicier - La Casnière (Corps de ferme) Angrie - 1.0
414971903 - sci caroline - 8 Place Victor Hugo 62500 Saint-Omer - 1.0
414971960 - sci v.o.f. - 5 Avenue de la Gare 10600 Payns - 1.0
414971986 - david robert - Rue du Four 56190 Le Guerno - 1.0
414971952 - anne bruneau - 23 Rue de Châtillon 35000 Rennes - 1.0
414971945 - jean-jacques marcotte - Zone à Lou Camp Ferrat 83120 Sainte-Maxime - 1.0


#### Recherche siret

In [21]:
search_es("41497192900040")

Identifiant non trouvé
Recherche nom
987.99896 - 414971929 - financiere ganymede - Rue Rigoberta Menchu 84000 Avignon - 4.0


#### Recherche siret partiel

In [22]:
search_es("41497192900")

414971929 - financiere ganymede - 60 Avenue du Général de Gaulle 92800 Puteaux - 4.0
414971929 - financiere ganymede - Rue Rigoberta Menchu 84000 Avignon - 4.0
414971929 - financiere ganymede - 26 Boulevard Saint-roch 84000 Avignon - 4.0
414971929 - financiere ganymede - 4 Chemin des Sablons 78160 Marly-le-Roi - 4.0


#### Recherche mixte nom + siren

In [23]:
search_es("ganymede 383657467")

Identifiant non trouvé
Recherche nom
188.51324 - 423208180 - ganymede - Route de Gruissan 11100 Narbonne - 1.0
180.70328 - 409716305 - ganymede - 36 Rue Etienne Ganneron 77600 Bussy-Saint-Georges - 1.0
180.70328 - 901723403 - ganymede - 58 Boulevard Aristide Briand 63000 Clermont-Ferrand - 1.0
180.70096 - 880878145 - ganymede - 128 Rue la Boétie 75008 Paris - 1.0
180.70096 - 817883150 - ganymede - 3 Rue de la Gaîté 75014 Paris - 1.0
175.8141 - 803657048 - ganymede - 4 Impasse Bel Air 57970 Yutz - 1.0
170.7807 - 510973431 - ganymede - Traverse des Pénitents Noirs 13600 La Ciotat - 2.0
164.94908 - 402591820 - ganimede - 5 Rue Paul Gaffarel 21850 Saint-Apollinaire - 1.0
158.11334 - 442743209 - ganimede - Rue du Docteur David 88300 Neufchâteau - 1.0
158.11334 - 484284799 - ganimede - 115 Boulevard de la Bataille de Stalingrad 69100 Villeurbanne - 1.0


#### Recherche mixte nom + siret

In [24]:
search_es("ganymede 38365746700023")

Identifiant non trouvé
Recherche nom
188.51324 - 423208180 - ganymede - Route de Gruissan 11100 Narbonne - 1.0
180.70328 - 409716305 - ganymede - 36 Rue Etienne Ganneron 77600 Bussy-Saint-Georges - 1.0
180.70328 - 901723403 - ganymede - 58 Boulevard Aristide Briand 63000 Clermont-Ferrand - 1.0
180.70096 - 880878145 - ganymede - 128 Rue la Boétie 75008 Paris - 1.0
180.70096 - 817883150 - ganymede - 3 Rue de la Gaîté 75014 Paris - 1.0
175.8141 - 803657048 - ganymede - 4 Impasse Bel Air 57970 Yutz - 1.0
170.7807 - 510973431 - ganymede - Traverse des Pénitents Noirs 13600 La Ciotat - 2.0
164.94908 - 402591820 - ganimede - 5 Rue Paul Gaffarel 21850 Saint-Apollinaire - 1.0
158.11334 - 442743209 - ganimede - Rue du Docteur David 88300 Neufchâteau - 1.0
158.11334 - 484284799 - ganimede - 115 Boulevard de la Bataille de Stalingrad 69100 Villeurbanne - 1.0


#### Recherche par numéro association

In [25]:
search_es("l'amicale W751188429")

Identifiant non trouvé
Recherche nom
89.0826 - 840841498 - l'amicale - 34 Route d'Alayrac 12500 Espalion - 1.0
89.0826 - 531825677 - l'amicale - 9 Rue des Trois Bornes 75011 Paris - 1.0
88.898415 - 520654898 - l'amicale - Place de la Mairie 08460 Thin-le-Moutier - 1.0
88.79974 - 398694034 - l'amicale - 1 Place Royale 44000 Nantes - 3.0
88.79974 - 798574075 - l'amicale - 75 Les Portes Domitiennes 34740 Vendargues - 1.0
88.18651 - 492899257 - l'amicale - 130 Rue de Lodi 13006 Marseille - 1.0
88.18651 - 520651860 - l'amicale - 3 Place Ducale 08000 Charleville-Mézières - 1.0
88.18651 - 752555219 - l'amicale - 7 Rue de la Bonneterie 33640 Portets - 1.0
78.79381 - 844471565 - l'amirale - 23 Rue Jean-jacques Rousseau 75001 Paris - 1.0
73.22195 - 805383080 - l'amicale broussoise - 1.0


In [26]:
search_es("W751188429")

Identifiant non trouvé
Recherche nom
383.22662 - 509964938 - l'amicale du grand chauve - 23 Rue Théodore Deck 75015 Paris - 1.0


In [27]:
search_es("W7511884")

401140165 - visages du saxophone - 6 Impasse Saint-sébastien 75011 Paris - 6.0
448778134 - compagnie de vent d est - 22 Rue Deparcieux 75014 Paris - 4.0
508822244 - brasil-ao-vivo - 2 Rue Albert Camus 75010 Paris - 3.0
490483278 - la mome perchee - 36 Rue Hoche 93500 Pantin - 3.0
517585469 - world inline skaters association (wisa) - 51 Rue de la Glacière 75013 Paris - 3.0
509881850 - la compagnie raymond acquaviva - 34 Rue Marcadet 75018 Paris - 3.0
520339359 - association d aide a la danse production (a.a.d production) - 4 b Cité Véron 75018 Paris - 2.0
512075870 - organisation de solidarite internationale des migrants hexagone sud (osim hs) - 14 Avenue René Boylesve 75016 Paris - 2.0
509796280 - ass la fabrique des echos - 15 Rue des Gelinieres 85200 Pissotte - 2.0
509584447 - association du theatre de la liane - 181 Avenue Daumesnil 75012 Paris - 2.0


#### Recherche erreurs

In [28]:
search_es("amicale du grand chave")

Identifiant non trouvé
Recherche nom
149.55905 - 509964938 - l'amicale du grand chauve - 23 Rue Théodore Deck 75015 Paris - 1.0
127.90685 - 850682659 - grand est chape - Rue du Jardin Marque 54670 Millery - 2.0
127.813126 - 432606473 - chabe grand sud - 13 Rue Latour-maubourg 06400 Cannes - 9.0
123.89011 - 898857511 - la grand cave - Grande Rue 51480 Belval-sous-Châtillon - 1.0
115.36351 - 419044458 - chave - 37 Rue Frederic Mistral 13400 Aubagne - 1.0
115.12944 - 897671079 - chave - 1250 Chemin de la Chave 07270 Gilhoc-sur-Ormèze - 1.0
111.03165 - 837745652 - sci du grand chauve - 30 Rue Clément Marot 80480 Salouël - 1.0
109.17982 - 786021576 - amicale laique grand carcouet - Rue Louis Brisset 44100 Nantes - 1.0
109.16191 - 509515102 - amicale basket grand est - 25 Rue de Venise 54500 Vandœuvre-lès-Nancy - 1.0
109.16191 - 500358940 - amicale le grand verger - Rue de la Maurelle 13013 Marseille - 1.0


In [29]:
search_es("financiere ganymeda")

Identifiant non trouvé
Recherche nom
198.48743 - 414971929 - financiere ganymede - Rue Rigoberta Menchu 84000 Avignon - 4.0
164.94908 - 423208180 - ganymede - Route de Gruissan 11100 Narbonne - 1.0
158.11536 - 409716305 - ganymede - 36 Rue Etienne Ganneron 77600 Bussy-Saint-Georges - 1.0
158.11536 - 901723403 - ganymede - 58 Boulevard Aristide Briand 63000 Clermont-Ferrand - 1.0
158.11334 - 880878145 - ganymede - 128 Rue la Boétie 75008 Paris - 1.0
158.11334 - 817883150 - ganymede - 3 Rue de la Gaîté 75014 Paris - 1.0
153.83734 - 803657048 - ganymede - 4 Impasse Bel Air 57970 Yutz - 1.0
149.4331 - 510973431 - ganymede - Traverse des Pénitents Noirs 13600 La Ciotat - 2.0
141.38492 - 402591820 - ganimede - 5 Rue Paul Gaffarel 21850 Saint-Apollinaire - 1.0
141.38492 - 443555974 - danimeda - 3.0


In [30]:
search_es("finaniere anymeda")

Identifiant non trouvé
Recherche nom
167.52072 - 414971929 - financiere ganymede - Rue Rigoberta Menchu 84000 Avignon - 4.0
129.08589 - 423208180 - ganymede - Route de Gruissan 11100 Narbonne - 1.0
129.08589 - 443555974 - danimeda - 3.0
129.08589 - 402243786 - animedia - 15 Rue Traversière 75012 Paris - 4.0
129.08589 - 494453459 - alameda - 15 Rue Jean Moulin 33310 Lormont - 1.0
129.07214 - 835362997 - lanameda - 40 Boulevard Joseph Vernet 13008 Marseille - 1.0
129.07214 - 880878145 - ganymede - 128 Rue la Boétie 75008 Paris - 1.0
129.07214 - 492809934 - anareda - 5 Rue Bonaparte 20000 Ajaccio - 2.0
129.07214 - 817883150 - ganymede - 3 Rue de la Gaîté 75014 Paris - 1.0
127.173546 - 409716305 - ganymede - 36 Rue Etienne Ganneron 77600 Bussy-Saint-Georges - 1.0


In [31]:
search_es("juste ebeniste")

Identifiant non trouvé
Recherche nom
232.4116 - 824024350 - juste ebeniste - 14 Rue de Cornen 44510 Le Pouliguen - 4.0
130.50362 - 384117545 - ecorce ebeniste - 1000 Route de Vonnas 01380 Saint-Cyr-sur-Menthon - 1.0
130.50362 - 818118051 - pb ebeniste - 4 Rue du Carreau 69960 Corbas - 2.0
130.50362 - 843490061 - rb ebeniste - 105 Chemin de la Garenne 69210 Sain-Bel - 1.0
128.38966 - 415283100 - un ebeniste - 50 Rue Anthime Ravoire 13300 Salon-de-Provence - 1.0
124.258835 - 901754226 - juste - 1.0
124.02732 - 890154347 - juste - 10 Cours Pierre Puget 13006 Marseille - 1.0
123.45861 - 825208994 - juste - 119 Cours de la République 33470 Gujan-Mestras - 3.0
123.36073 - 490627031 - juste - 1.0
123.36073 - 841258502 - juste - 60 Avenue Centrale 07380 Lalevade-d'Ardèche - 1.0


In [32]:
search_es("quacks")

Identifiant non trouvé
Recherche nom
3414.265 - 890071210 - quacks - 9 bis Rue de la Guadeloupe 75018 Paris - 1.0
138.06671 - 440649416 - quick et quack - Saut Sabbat 97360 Mana - 1.0
118.33005 - 820978799 - quarks - 77 Rue des Plantes 75014 Paris - 1.0
117.9064 - 814066767 - quarks - Section Vinty 97115 Sainte-Rose - 1.0
116.27124 - 891710865 - quarks - 87 Rue Colbert 92700 Colombes - 1.0
97.351875 - 823559034 - agence quarks - 36 Rue Saint-maur 75011 Paris - 2.0
97.351875 - 819723693 - quarks services - 48 Grande-rue 25000 Besançon - 3.0
97.26609 - 894821966 - quicks location - 33 Avenue de Marboz 01000 Bourg-en-Bresse - 1.0
97.26609 - 389270943 - compagnie quarks - 12 Cité Dupetit-thouars 75003 Paris - 1.0
97.25301 - 818505950 - creativity quarks - 323 La Cabriere 83690 Salernes - 1.0


In [33]:
search_es("Qwant")

Identifiant non trouvé
Recherche nom
10651.473 - 532867256 - qwant - 62 Boulevard Victor Hugo 92200 Neuilly-sur-Seine - 11.0
3947.9429 - 834481426 - qwant enterprise - 7 Rue Spontini 75016 Paris - 2.0
2828.63 - 830268140 - qwant music - Rue Bonaparte 20000 Ajaccio - 1.0
2827.7322 - 838464162 - qwant pay - 7 Rue Spontini 75016 Paris - 1.0
137.80418 - 891257131 - qwantx - 121 Rue d'Aguesseau 92100 Boulogne-Billancourt - 1.0
136.63788 - 791901630 - quant - 3 Rue du Helder 64200 Biarritz - 5.0
133.72453 - 803089614 - qwart - 8 Chemin des Mignonades 81710 Saïx - 1.0
133.72453 - 877957860 - qwanty - 13 Boulevard René Levasseur 72000 Le Mans - 3.0
132.92368 - 882797558 - quant - 7 Rue de Normandie 53410 Saint-Ouën-des-Toits - 1.0
132.92368 - 899075089 - qwanta - 66 Rue Victor Hugo 78420 Carrières-sur-Seine - 1.0


In [34]:
search_es("suneris")

Identifiant non trouvé
Recherche nom
6717.6475 - 811818848 - suneris solution - 6 Rue Dewoitine 78140 Vélizy-Villacoublay - 6.0
133.50772 - 844217216 - muneris - 1.0
132.75339 - 810532861 - luneris - 2 Rue de l'Appel du 18 Juin 92800 Puteaux - 1.0
132.75339 - 422001875 - funeris - 97 Rue du Terme la Ville 51460 Courtisols - 3.0
132.75339 - 812555332 - sunergis - 4 Chemin du Solarium 33170 Gradignan - 2.0
130.79538 - 791846637 - sunesis - 52 Rue du Capitaine Guynemer 92400 Courbevoie - 1.0
130.19247 - 833821150 - soneris - 49 Rue de Ponthieu 75008 Paris - 1.0
130.19247 - 383475530 - syneris - 2 Rue Baudoin 75013 Paris - 8.0
129.65622 - 394217061 - funeris - 40 Rue des Aulnes 92330 Sceaux - 1.0
111.25645 - 410303481 - generis - 28 Boulevard de Pesaro 92000 Nanterre - 33.0


In [35]:
search_es("jordan guedj")

Identifiant non trouvé
Recherche nom
196.21658 - 803996461 - jordan guedj - Avenue William Booth 13011 Marseille - 1.0
149.88435 - 513549956 - jordan bogdan - 46 Route de Metz 57100 Thionville - 1.0
132.58118 - 448731968 - guedj - 50 Boulevard Froissart 59300 Valenciennes - 1.0
119.91124 - 795273580 - jordan jordan - Résidence les Floralies (Maison de retraite) Le Pontet - 1.0
117.393845 - 428820856 - scp guedj (scp guedj) - 1 Rue Cristino Garcia 95600 Eaubonne - 1.0
110.86404 - 887857910 - laurine guedj - Rue de la Planchette (Chemin) Saulcy-sur-Meurthe - 1.0
110.86404 - 750535601 - julie guedj - 50 Rue de Longchamp 92200 Neuilly-sur-Seine - 2.0
110.86404 - 830484929 - benjamin guedj - 15 Quai de Stalingrad 92100 Boulogne-Billancourt - 2.0
110.86404 - 885047795 - richard guedj - 98 Rue des Fontenelles 72000 Le Mans - 1.0
110.86404 - 482229622 - yoni guedj - 9 Impasse Charles de Coulomb 31830 Plaisance-du-Touch - 3.0
